In [1]:
import numpy as np
import pandas as pd
import os
import re
import shutil

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.utils import to_categorical, image_dataset_from_directory
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import KFold

from collections import defaultdict
from glob import glob
import random

import cv2
from PIL import Image
matplotlib.use('Agg')   # type: ignore
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [2]:
# Define dataset directory
images_dir = "combined-dataset"
# "C:/Users/theof/OneDrive/Documents/Github/genome_color_unpickler/train-raw/arrays"

# Training parameters
img_size = (32, 32)
batch_size = 32
epochs = 50

# Dictionary to store accuracies
max_accuracies = {}

In [3]:
# Function to build a simple CNN model
def cnn_model(input_shape, num_classes):
	model = Sequential([
		Conv2D(32, (5,5), activation='relu', input_shape=input_shape),
		MaxPooling2D(2,2),
		Conv2D(64, (5,5), activation='relu'),
		MaxPooling2D(2,2),
		Flatten(),
		Dense(128, activation='relu'),
		Dense(num_classes, activation='softmax')  # Adjust output neurons based on classes
	])
	model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
	return model

In [4]:
# Dataset Preparation and Undersampling
def prepare_dataset(method_path):
	class_dirs = [d for d in os.listdir(method_path) if os.path.isdir(os.path.join(method_path, d))]
	class_to_images = defaultdict(list)
	for class_name in class_dirs:
		class_path = os.path.join(method_path, class_name)
		image_paths = glob(os.path.join(class_path, "*"))
		class_to_images[class_name].extend(image_paths)

	target_size = min(
		max(1000 if k in ['Euro-American', 'East Asian'] else len(v), len(v))
		for k, v in class_to_images.items()
	)
	balanced_image_paths = []
	balanced_labels = []
	for cls, imgs in class_to_images.items():
		sampled = random.sample(imgs, target_size) if len(imgs) > target_size else imgs
		balanced_image_paths.extend(sampled)
		balanced_labels.extend([cls] * len(sampled))

	combined = list(zip(balanced_image_paths, balanced_labels))
	random.shuffle(combined)
	balanced_image_paths, balanced_labels = zip(*combined)
	df = pd.DataFrame({'filename': balanced_image_paths, 'class': balanced_labels})
	return df

In [6]:
# Iterate through resolution folders
for resolution_folder in sorted(os.listdir(images_dir), key=lambda x: int(re.search(r'\d+', x).group())):
	resolution_path = os.path.join(images_dir, resolution_folder)

	if os.path.isdir(resolution_path):
		# Iterate through method folders inside resolution
		for method_folder in os.listdir(resolution_path):
			method_path = os.path.join(resolution_path, method_folder)

			if os.path.isdir(method_path):
				print(f"\nTraining {resolution_folder}/{method_folder}...")
    
				# Prepare dataset
				df = prepare_dataset(method_path)

				# Data Preprocessing
				datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

				# Use subset='training' and 'validation' for split
				train_data = datagen.flow_from_dataframe(
					dataframe=df,
					target_size=img_size,
					batch_size=batch_size,
					class_mode='categorical',
					subset='training'
				)
				val_data = datagen.flow_from_dataframe(
					dataframe=df,
					target_size=img_size,
					batch_size=batch_size,
					class_mode='categorical',
					subset='validation',
					shuffle=False
				)

				num_samples = len(df)
				num_classes = len(df['class'].unique())
				input_shape = (img_size[0], img_size[1], 3)

				# Initialize and train the model
				model = cnn_model(input_shape, num_classes)
				history = model.fit(
					train_data,
					validation_data=val_data,
					epochs=epochs,
					verbose=1
				)

				# Save the model
				model_path = os.path.join("models/standard", resolution_folder, method_folder)
				os.makedirs(model_path, exist_ok=True)
				model_name = f"{resolution_folder}_{method_folder}_mask_5_aug-under.h5"
				model.save(os.path.join(model_path, model_name))
				print(f"Model saved to {model_path}")

				# Store final validation accuracy
				final_val_acc = max(history.history['val_accuracy'])
				key = f"{resolution_folder} - {method_folder}"
				max_accuracies[key] = final_val_acc
				print(f"Validation accuracy for {key}: {final_val_acc:.4f}")



Training 4px/Chargaff-Composante-Diversite...
Found 1668 validated image filenames belonging to 5 classes.
Found 417 validated image filenames belonging to 5 classes.
Epoch 1/50
53/53 [==============================] - 2s 23ms/step - loss: 1.6150 - accuracy: 0.2188 - val_loss: 1.6158 - val_accuracy: 0.1894
Epoch 2/50
53/53 [==============================] - 1s 19ms/step - loss: 1.6037 - accuracy: 0.2332 - val_loss: 1.5942 - val_accuracy: 0.2110
Epoch 3/50
53/53 [==============================] - 1s 17ms/step - loss: 1.5672 - accuracy: 0.2818 - val_loss: 1.5425 - val_accuracy: 0.2974
Epoch 4/50
53/53 [==============================] - 1s 17ms/step - loss: 1.5308 - accuracy: 0.3004 - val_loss: 1.5472 - val_accuracy: 0.2398
Epoch 5/50
53/53 [==============================] - 1s 17ms/step - loss: 1.5141 - accuracy: 0.3339 - val_loss: 1.5295 - val_accuracy: 0.2662
Epoch 6/50
53/53 [==============================] - 1s 16ms/step - loss: 1.5032 - accuracy: 0.3255 - val_loss: 1.5101 - val_acc

c:\Users\theof\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Found 1668 validated image filenames belonging to 5 classes.
Found 417 validated image filenames belonging to 5 classes.
Epoch 1/50
53/53 [==============================] - 2s 22ms/step - loss: 1.6092 - accuracy: 0.2182 - val_loss: 1.5791 - val_accuracy: 0.2422
Epoch 2/50
53/53 [==============================] - 1s 17ms/step - loss: 1.5648 - accuracy: 0.2440 - val_loss: 1.5179 - val_accuracy: 0.2542
Epoch 3/50
53/53 [==============================] - 1s 17ms/step - loss: 1.5387 - accuracy: 0.2422 - val_loss: 1.5207 - val_accuracy: 0.2686
Epoch 4/50
53/53 [==============================] - 1s 17ms/step - loss: 1.5105 - accuracy: 0.2680 - val_loss: 1.5341 - val_accuracy: 0.2614
Epoch 5/50
53/53 [==============================] - 1s 17ms/step - loss: 1.5155 - accuracy: 0.2860 - val_loss: 1.5173 - val_accuracy: 0.2782
Epoch 6/50
53/53 [==============================] - 1s 17ms/step - loss: 1.5087 - accuracy: 0.2938 - val_loss: 1.5270 - val_accuracy: 0.2494
Epoch 7/50
53/53 [===============

c:\Users\theof\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Found 1668 validated image filenames belonging to 5 classes.
Found 417 validated image filenames belonging to 5 classes.
Epoch 1/50
53/53 [==============================] - 2s 21ms/step - loss: 1.5952 - accuracy: 0.2368 - val_loss: 1.6631 - val_accuracy: 0.2494
Epoch 2/50
53/53 [==============================] - 1s 16ms/step - loss: 1.5165 - accuracy: 0.2890 - val_loss: 1.4741 - val_accuracy: 0.3357
Epoch 3/50
53/53 [==============================] - 1s 16ms/step - loss: 1.5034 - accuracy: 0.3094 - val_loss: 1.4918 - val_accuracy: 0.3237
Epoch 4/50
53/53 [==============================] - 1s 18ms/step - loss: 1.4559 - accuracy: 0.3399 - val_loss: 1.4657 - val_accuracy: 0.3573
Epoch 5/50
53/53 [==============================] - 1s 20ms/step - loss: 1.4646 - accuracy: 0.3435 - val_loss: 1.4583 - val_accuracy: 0.3381
Epoch 6/50
53/53 [==============================] - 1s 20ms/step - loss: 1.4361 - accuracy: 0.3579 - val_loss: 1.5051 - val_accuracy: 0.3333
Epoch 7/50
53/53 [===============

...

c:\Users\theof\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Found 1668 validated image filenames belonging to 5 classes.
Found 417 validated image filenames belonging to 5 classes.
Epoch 1/50
53/53 [==============================] - 2s 23ms/step - loss: 1.5163 - accuracy: 0.3052 - val_loss: 1.3454 - val_accuracy: 0.5012
Epoch 2/50
53/53 [==============================] - 1s 17ms/step - loss: 1.2944 - accuracy: 0.4508 - val_loss: 1.2096 - val_accuracy: 0.5635
Epoch 3/50
53/53 [==============================] - 1s 16ms/step - loss: 1.0638 - accuracy: 0.5665 - val_loss: 0.9858 - val_accuracy: 0.5612
Epoch 4/50
53/53 [==============================] - 1s 17ms/step - loss: 0.8370 - accuracy: 0.6876 - val_loss: 0.7249 - val_accuracy: 0.7362
Epoch 5/50
53/53 [==============================] - 1s 17ms/step - loss: 0.6590 - accuracy: 0.7686 - val_loss: 0.5931 - val_accuracy: 0.7866
Epoch 6/50
53/53 [==============================] - 1s 17ms/step - loss: 0.5769 - accuracy: 0.7986 - val_loss: 0.5849 - val_accuracy: 0.7794
Epoch 7/50
53/53 [===============

c:\Users\theof\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Found 1668 validated image filenames belonging to 5 classes.
Found 417 validated image filenames belonging to 5 classes.
Epoch 1/50
53/53 [==============================] - 2s 23ms/step - loss: 1.5689 - accuracy: 0.2548 - val_loss: 1.4977 - val_accuracy: 0.3549
Epoch 2/50
53/53 [==============================] - 1s 17ms/step - loss: 1.3902 - accuracy: 0.3999 - val_loss: 1.3003 - val_accuracy: 0.4532
Epoch 3/50
53/53 [==============================] - 1s 17ms/step - loss: 1.2586 - accuracy: 0.4688 - val_loss: 1.0655 - val_accuracy: 0.6547
Epoch 4/50
53/53 [==============================] - 1s 16ms/step - loss: 1.0422 - accuracy: 0.5995 - val_loss: 1.0423 - val_accuracy: 0.4916
Epoch 5/50
53/53 [==============================] - 1s 17ms/step - loss: 0.8402 - accuracy: 0.6984 - val_loss: 0.7028 - val_accuracy: 0.7266
Epoch 6/50
53/53 [==============================] - 1s 17ms/step - loss: 0.6892 - accuracy: 0.7644 - val_loss: 0.5980 - val_accuracy: 0.8010
Epoch 7/50
53/53 [===============

c:\Users\theof\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [7]:
# Create positions for bars
num_bars = len(max_accuracies)
x_positions = np.arange(num_bars)

# Add extra space every 10 bars
for i in range(10, num_bars, 10):  
	x_positions[i:] += 1  # Shift everything after every 10th bar

plt.figure(figsize=(15, 10))
plt.bar(x_positions, max_accuracies.values(), color='blue', width=0.5)
plt.grid(axis='y', linestyle='--', alpha=0.5)

# Custom x-ticks for readability
plt.xticks(x_positions, max_accuracies.keys(), rotation=90)

plt.yticks(np.arange(0, 1.1, 0.2))

plt.ylabel("Validation Accuracy")
plt.title("CNN Validation Results")
plt.tight_layout()
plt.show()

# plt.savefig("../../imgs/graphs/kfold/cnn_validation_accuracy_bar_mask_5-kfold_aug2.png")

C:\Users\theof\AppData\Local\Temp\ipykernel_38040\22334614.py:21: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [8]:
# Extract and sort resolutions numerically
resolutions = sorted(set(k.split(" - ")[0] for k in max_accuracies.keys()), key=lambda x: int(x.replace("px", "")))
methods = sorted(set(k.split(" - ")[1] for k in max_accuracies.keys()))

# Organize data for plotting
data = {method: [max_accuracies.get(f"{res} - {method}", None) for res in resolutions] for method in methods}

# Define a color map for better distinction
colors = plt.cm.get_cmap("tab10", len(methods))  # Use "tab10" color map with enough colors

# Plot the lines
plt.figure(figsize=(16, 10))
for i, (method, accuracies) in enumerate(data.items()):
	plt.plot(resolutions, accuracies, marker="o", linestyle="-", linewidth=2, markersize=8, label=method, color=colors(i))

# Labels and title
plt.xlabel("Resolution", fontsize=12)
plt.ylabel("Max Accuracy", fontsize=12)
plt.title("CNN Validation Accuracy by Resolution and Method", fontsize=14)

# Move legend outside the plot for better clarity
plt.legend(title="Method", bbox_to_anchor=(1.05, 1), loc="upper left", fontsize=10)

# Add a grid with transparency
plt.grid(True, linestyle="--", alpha=0.6)

# Improve layout to fit legend properly
plt.tight_layout()

# Show plot
plt.show()

plt.savefig("../../imgs/graphs/standard-aug/cnn_validation_accuracy_mask_5_aug-under.png")

C:\Users\theof\AppData\Local\Temp\ipykernel_38040\676422172.py:9: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  colors = plt.cm.get_cmap("tab10", len(methods))  # Use "tab10" color map with enough colors
C:\Users\theof\AppData\Local\Temp\ipykernel_38040\676422172.py:31: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [9]:
# Extract and sort resolutions numerically
resolutions = sorted(set(k.split(" - ")[0] for k in max_accuracies.keys()), key=lambda x: int(x.replace("px", "")))

# Extract unique methods and split into two groups of 5
methods = sorted(set(k.split(" - ")[1] for k in max_accuracies.keys()))
methods_group1 = methods[:5]
methods_group2 = methods[5:]

# Organize data for plotting
data1 = {method: [max_accuracies.get(f"{res} - {method}", None) for res in resolutions] for method in methods_group1}
data2 = {method: [max_accuracies.get(f"{res} - {method}", None) for res in resolutions] for method in methods_group2}

# Create subplots
fig, axs = plt.subplots(2, 1, figsize=(10, 10), sharex=True, sharey=True)

colors = plt.cm.get_cmap("tab10", len(methods_group1))  # Use "tab10" color map with enough colors

# Plot first group
for method, accuracies in data1.items():
	axs[0].plot(resolutions, accuracies, marker="o", label=method, color=colors(methods_group1.index(method)))
axs[0].set_title("CNN Validation Accuracy (Group 1)")
axs[0].set_ylabel("Max Accuracy")
axs[0].legend(title="Method")
axs[0].grid(True, linestyle="--", alpha=0.6)

# Plot second group
for method, accuracies in data2.items():
	axs[1].plot(resolutions, accuracies, marker="o", label=method, color=colors(methods_group2.index(method)))
axs[1].set_title("CNN Validation Accuracy (Group 2)")
axs[1].set_xlabel("Resolution")
axs[1].set_ylabel("Max Accuracy")
axs[1].legend(title="Method")
axs[1].grid(True, linestyle="--", alpha=0.6)

# Adjust layout
plt.tight_layout()
plt.show()

plt.savefig("../../imgs/graphs/standard-aug/cnn_validation_accuracy_groups_mask_5_aug-under.png")

C:\Users\theof\AppData\Local\Temp\ipykernel_38040\2915024216.py:16: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  colors = plt.cm.get_cmap("tab10", len(methods_group1))  # Use "tab10" color map with enough colors
C:\Users\theof\AppData\Local\Temp\ipykernel_38040\2915024216.py:37: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [10]:
# confusion matrix
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
from keras.models import load_model

# Load the model
model_path = "models/standard/100px/Chargaff-Composante-Diversite/100px_Chargaff-Composante-Diversite_mask_5_aug-under.h5"
model = load_model(model_path)
model.summary()

# Load the test data
test_data_dir = 'C:/Users/theof/OneDrive/Documents/Github/genome_color_unpickler/test-raw/arrays/100px/Chargaff-Composante-Diversite'

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
	test_data_dir,
	target_size=img_size,
	batch_size=batch_size,
	class_mode='categorical',
	shuffle=False
)
# Get the true labels
true_labels = test_generator.classes
class_labels = list(test_generator.class_indices.keys())

# Get the predicted labels
predictions = model.predict(test_generator, steps=len(test_generator), verbose=1)
predicted_labels = np.argmax(predictions, axis=1)
# Generate the confusion matrix
cm = confusion_matrix(true_labels, predicted_labels)

# Plot the confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.title('Confusion Matrix at 100px Resolution for Chargaff-Composante-Diversite Method')
plt.tight_layout()
plt.show()

plt.savefig("../../imgs/graphs/standard-aug/cnn_confusion_matrix_100px_mask_5_aug-under.png")

Model: "sequential_40"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_80 (Conv2D)          (None, 28, 28, 32)        2432      
                                                                 
 max_pooling2d_80 (MaxPooli  (None, 14, 14, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_81 (Conv2D)          (None, 10, 10, 64)        51264     
                                                                 
 max_pooling2d_81 (MaxPooli  (None, 5, 5, 64)          0         
 ng2D)                                                           
                                                                 
 flatten_40 (Flatten)        (None, 1600)              0         
                                                                 
 dense_80 (Dense)            (None, 128)             

C:\Users\theof\AppData\Local\Temp\ipykernel_38040\2898753474.py:39: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [16]:
# Define dataset directory
mosaics_dir = 'combined-dataset-mosaics'
# "C:/Users/theof/OneDrive/Documents/Github/genome_color_unpickler/train-raw/mosaics"

# Training parameters
img_size = (20, 50)  # Resize images to 64x64
batch_size = 32
epochs = 50
 
# Dictionary to store max accuracies
max_accuracies_mos = {}

In [17]:
# Mosaic version (without cross-validation)
# Iterate through resolution folders (e.g., 4px, 16px)
for resolution_folder in sorted(os.listdir(mosaics_dir), key=lambda x: int(re.search(r'\d+', x).group())):
	resolution_path = os.path.join(mosaics_dir, resolution_folder)

	if os.path.isdir(resolution_path):
		print(f"\nTraining {resolution_folder}...")

		# Data Preprocessing
		datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
  
		# Prepare dataset
		new_df = prepare_dataset(resolution_path)

		# Create training and validation generators
		train_data = datagen.flow_from_dataframe(
			dataframe=new_df,
			target_size=img_size,
			batch_size=batch_size,
			class_mode='categorical',
			subset='training'
		)
		val_data = datagen.flow_from_dataframe(
			dataframe=new_df,
			target_size=img_size,
			batch_size=batch_size,
			class_mode='categorical',
			subset='validation',
			shuffle=False
		)

		num_samples = len(new_df)
		num_classes = len(new_df['class'].unique())
		input_shape = (img_size[0], img_size[1], 3)

		# Initialize and train model
		model = cnn_model(input_shape, num_classes)
		history = model.fit(
			train_data,
			validation_data=val_data,
			epochs=epochs,
			verbose=1
		)

		# Save the model
		model_path = os.path.join("models/standard-mosaic", resolution_folder)
		os.makedirs(model_path, exist_ok=True)
		model_name = f"{resolution_folder}_mosaic_mask_5_aug-under.h5"
		model.save(os.path.join(model_path, model_name))
		print(f"Model saved to {model_path}")

		# Store final validation accuracy
		final_val_acc = max(history.history['val_accuracy'])
		max_accuracies_mos[resolution_folder] = final_val_acc
		print(f"Validation accuracy for {resolution_folder}: {final_val_acc:.4f}")


Training 4px...
Found 888 validated image filenames belonging to 5 classes.
Found 222 validated image filenames belonging to 5 classes.
Epoch 1/50
28/28 [==============================] - 1s 26ms/step - loss: 1.6240 - accuracy: 0.2027 - val_loss: 1.6152 - val_accuracy: 0.1802
Epoch 2/50
28/28 [==============================] - 1s 18ms/step - loss: 1.6093 - accuracy: 0.2050 - val_loss: 1.6169 - val_accuracy: 0.1486
Epoch 3/50
28/28 [==============================] - 1s 18ms/step - loss: 1.6042 - accuracy: 0.2297 - val_loss: 1.6092 - val_accuracy: 0.1622
Epoch 4/50
28/28 [==============================] - 0s 17ms/step - loss: 1.5963 - accuracy: 0.2376 - val_loss: 1.5979 - val_accuracy: 0.2162
Epoch 5/50
28/28 [==============================] - 0s 17ms/step - loss: 1.5749 - accuracy: 0.2984 - val_loss: 1.5884 - val_accuracy: 0.2297
Epoch 6/50
28/28 [==============================] - 1s 18ms/step - loss: 1.5619 - accuracy: 0.2827 - val_loss: 1.5550 - val_accuracy: 0.2568
Epoch 7/50
28/28 

c:\Users\theof\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Found 224 validated image filenames belonging to 5 classes.
Epoch 1/50
28/28 [==============================] - 1s 30ms/step - loss: 1.6195 - accuracy: 0.1998 - val_loss: 1.6023 - val_accuracy: 0.2054
Epoch 2/50
28/28 [==============================] - 1s 19ms/step - loss: 1.5686 - accuracy: 0.3225 - val_loss: 1.5585 - val_accuracy: 0.2946
Epoch 3/50
28/28 [==============================] - 0s 17ms/step - loss: 1.5312 - accuracy: 0.3315 - val_loss: 1.4757 - val_accuracy: 0.4152
Epoch 4/50
28/28 [==============================] - 0s 17ms/step - loss: 1.4376 - accuracy: 0.4342 - val_loss: 1.4154 - val_accuracy: 0.4375
Epoch 5/50
28/28 [==============================] - 0s 16ms/step - loss: 1.3096 - accuracy: 0.4810 - val_loss: 1.2859 - val_accuracy: 0.4598
Epoch 6/50
28/28 [==============================] - 0s 17ms/step - loss: 1.1930 - accuracy: 0.5268 - val_loss: 1.4028 - val_accuracy: 0.4464
Epoch 7/50
28/28 [==============================] - 1s 18ms/step - loss: 1.1023 - accuracy: 0.

c:\Users\theof\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Found 896 validated image filenames belonging to 5 classes.
Found 224 validated image filenames belonging to 5 classes.
Epoch 1/50
28/28 [==============================] - 2s 34ms/step - loss: 1.5928 - accuracy: 0.2734 - val_loss: 1.5348 - val_accuracy: 0.2812
Epoch 2/50
28/28 [==============================] - 1s 23ms/step - loss: 1.4814 - accuracy: 0.3281 - val_loss: 1.4011 - val_accuracy: 0.4330
Epoch 3/50
28/28 [==============================] - 1s 21ms/step - loss: 1.2951 - accuracy: 0.4766 - val_loss: 1.2104 - val_accuracy: 0.4955
Epoch 4/50
28/28 [==============================] - 1s 18ms/step - loss: 1.1729 - accuracy: 0.5190 - val_loss: 1.1354 - val_accuracy: 0.5670
Epoch 5/50
28/28 [==============================] - 1s 24ms/step - loss: 1.0577 - accuracy: 0.5469 - val_loss: 1.0077 - val_accuracy: 0.6339
Epoch 6/50
28/28 [==============================] - 1s 22ms/step - loss: 0.8978 - accuracy: 0.6920 - val_loss: 0.9111 - val_accuracy: 0.6295
Epoch 7/50
28/28 [================

c:\Users\theof\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Found 224 validated image filenames belonging to 5 classes.
Epoch 1/50
28/28 [==============================] - 2s 33ms/step - loss: 1.6112 - accuracy: 0.2232 - val_loss: 1.5776 - val_accuracy: 0.4241
Epoch 2/50
28/28 [==============================] - 1s 25ms/step - loss: 1.5562 - accuracy: 0.3136 - val_loss: 1.4772 - val_accuracy: 0.3750
Epoch 3/50
28/28 [==============================] - 1s 22ms/step - loss: 1.4313 - accuracy: 0.4040 - val_loss: 1.4425 - val_accuracy: 0.3929
Epoch 4/50
28/28 [==============================] - 1s 25ms/step - loss: 1.2928 - accuracy: 0.5145 - val_loss: 1.2028 - val_accuracy: 0.5045
Epoch 5/50
28/28 [==============================] - 1s 18ms/step - loss: 1.0529 - accuracy: 0.6272 - val_loss: 0.8971 - val_accuracy: 0.5938
Epoch 6/50
28/28 [==============================] - 1s 18ms/step - loss: 0.8580 - accuracy: 0.6931 - val_loss: 0.8308 - val_accuracy: 0.6205
Epoch 7/50
28/28 [==============================] - 1s 24ms/step - loss: 0.7294 - accuracy: 0.

c:\Users\theof\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Found 888 validated image filenames belonging to 5 classes.
Found 222 validated image filenames belonging to 5 classes.
Epoch 1/50
28/28 [==============================] - 2s 34ms/step - loss: 1.6047 - accuracy: 0.2477 - val_loss: 1.5303 - val_accuracy: 0.3604
Epoch 2/50
28/28 [==============================] - 1s 23ms/step - loss: 1.4992 - accuracy: 0.3378 - val_loss: 1.4007 - val_accuracy: 0.4955
Epoch 3/50
28/28 [==============================] - 1s 24ms/step - loss: 1.3893 - accuracy: 0.4347 - val_loss: 1.2905 - val_accuracy: 0.6126
Epoch 4/50
28/28 [==============================] - 1s 22ms/step - loss: 1.3143 - accuracy: 0.4223 - val_loss: 1.2166 - val_accuracy: 0.4595
Epoch 5/50
28/28 [==============================] - 1s 24ms/step - loss: 1.1561 - accuracy: 0.5462 - val_loss: 1.1679 - val_accuracy: 0.5090
Epoch 6/50
28/28 [==============================] - 1s 20ms/step - loss: 1.0279 - accuracy: 0.6396 - val_loss: 0.8996 - val_accuracy: 0.6622
Epoch 7/50
28/28 [================

c:\Users\theof\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [18]:
# Create positions for bars
num_bars = len(max_accuracies_mos)
x_positions = np.arange(num_bars)

plt.figure(figsize=(15, 10))
plt.bar(x_positions, max_accuracies_mos.values(), color='blue', width=0.5)
plt.grid(axis='y', linestyle='--', alpha=0.5)

# Custom x-ticks for readability
plt.xticks(x_positions, max_accuracies_mos.keys(), rotation=90)

plt.yticks(np.arange(0, 1.1, 0.2))

plt.ylabel("Max Accuracy")
plt.title("CNN Validation Results")
plt.tight_layout()
plt.show()

# plt.savefig("../../imgs/graphs/standard_mosaic/cnn_validation_accuracy_mosaics_mask_5_std.png")

C:\Users\theof\AppData\Local\Temp\ipykernel_38040\2427635834.py:17: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [19]:
# Extract and sort resolutions numerically
resolutions = sorted(max_accuracies_mos.keys(), key=lambda x: int(x.replace("px", "")))

# Get max accuracy values in the correct order
accuracies = [max_accuracies_mos[res] for res in resolutions]

# Plot the accuracy per resolution
plt.figure(figsize=(12, 6))
plt.plot(resolutions, accuracies, marker="o", linestyle="-", linewidth=2, markersize=8, color="blue", label="Accuracy")

# Labels and title
plt.xlabel("Resolution", fontsize=12)
plt.ylabel("Max Accuracy", fontsize=12)
plt.title("CNN Validation Accuracy by Resolution", fontsize=14)

# Add data points on the plot
for i, acc in enumerate(accuracies):
    plt.text(resolutions[i], acc, f"{acc:.2f}", fontsize=10, ha="right")

# Grid and legend
plt.grid(True, linestyle="--", alpha=0.6)
plt.legend(loc="lower right")

# Save and show plot
plt.tight_layout()
plt.show()
plt.savefig("../../imgs/graphs/standard-mosaic-aug/cnn_validation_accuracy_mosaics_line_mask_5_aug-under.png")

C:\Users\theof\AppData\Local\Temp\ipykernel_38040\3261844415.py:26: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [20]:
# confusion matrix
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
from keras.models import load_model

# Load the model for the new version
new_model_path = "models/standard-mosaic/100px/100px_mosaic_mask_5_aug-under.h5"
new_model = load_model(new_model_path)
new_model.summary()

# Load the test data for the new version
new_test_data_dir = 'C:/Users/theof/OneDrive/Documents/Github/genome_color_unpickler/test-raw/mosaics/100px'

new_test_datagen = ImageDataGenerator(rescale=1./255)
new_test_generator = new_test_datagen.flow_from_directory(
	new_test_data_dir,
	target_size=(img_size[0], img_size[1]),
	batch_size=batch_size,
	class_mode='categorical',
	shuffle=False
)

# Get the true labels for the new version
new_true_labels = new_test_generator.classes
new_class_labels = list(new_test_generator.class_indices.keys())

# Get the predicted labels for the new version
new_predictions = new_model.predict(new_test_generator, steps=len(new_test_generator), verbose=1)
new_predicted_labels = np.argmax(new_predictions, axis=1)

# Generate the confusion matrix for the new version
new_cm = confusion_matrix(new_true_labels, new_predicted_labels)

# Plot the confusion matrix for the new version
plt.figure(figsize=(10, 8))
sns.heatmap(new_cm, annot=True, fmt='d', cmap='Blues', xticklabels=new_class_labels, yticklabels=new_class_labels)
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.title('Confusion Matrix for Mosaic images at 100px')
plt.tight_layout()
plt.show()

plt.savefig("../../imgs/graphs/standard-mosaic-aug/cnn_confusion_matrix_mosaics_100px_mask_5_aug-under.png")

Model: "sequential_59"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_118 (Conv2D)         (None, 16, 46, 32)        2432      
                                                                 
 max_pooling2d_118 (MaxPool  (None, 8, 23, 32)         0         
 ing2D)                                                          
                                                                 
 conv2d_119 (Conv2D)         (None, 4, 19, 64)         51264     
                                                                 
 max_pooling2d_119 (MaxPool  (None, 2, 9, 64)          0         
 ing2D)                                                          
                                                                 
 flatten_59 (Flatten)        (None, 1152)              0         
                                                                 
 dense_118 (Dense)           (None, 128)             

C:\Users\theof\AppData\Local\Temp\ipykernel_38040\4257786525.py:41: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
